## <b>Chapter 4.텍스트 벡터화(Text_Vectorization)</b>

- 자연어 처리에서는 기계가 문자를 이해 할 수 있도록 수치화해주는 과정이 반드시 필요하다.
- 텍스트 벡터화의 대표적인 방법 : 원-핫 인코딩(One-hot encoding), TF-IDF(빈도수 기반 텍스트/문서 벡터화), 단어 임베딩(Word Embedding)

### 1. 원-핫 인코딩(One-hot encoding)

- 인코딩 : 컴퓨터는 텍스트를 직접 처리하는게 아니라 숫자로 변환하여 처리한다.
   - 인코딩에는 텍스트를 정수로 변환하는 `정수 인코딩`과 원핫 벡터로 표현하는 `원핫 인코딩`이 있다.

- 정수 인코딩(Integer Encoding) : 자연어를 컴퓨터가 이해할 수 있는 숫자(정수) 형태로 인코딩하는 과정이다.

- 원-핫 인코딩 : 정수로 표현되었지만 실제로는 문자인 데이터를 기계가 인식할 수 있도록 버꿔주는 방법이다.
   - N개의 단어를 각각 N차원의 벡터로 표현하는 방식이다.
   - 단어에 해당되는 차원(인덱스)에 1을 넣고 나머지에는 0을 입력한다.
   - 원-핫 인코딩은 단어 또는 문자를 기준으로 벡터화가 가능하다.

- 정수 인코딩과 원-핫 인코딩(하나의 문장)

  <img src='images/encoding1.png' width='980px'>

#### 1) 정수 인코딩

##### ① okt 객체 생성

In [1]:
from konlpy.tag import Okt  

okt = Okt()  

##### ② 토큰화

In [2]:
tokens = okt.morphs("나는 자연어 처리를 배운다")  
print(tokens)

['나', '는', '자연어', '처리', '를', '배운다']


##### ③ 정수 인코딩 : 형태소별 인덱스로 번호 부여

In [3]:
word_to_index = {word : index for index, word in enumerate(tokens)}
print('정수 인코딩 :',word_to_index)

정수 인코딩 : {'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


#### 2) 원-핫 인코딩

##### ① 원-핫 벡터 생성 함수 정의

In [4]:
def one_hot_encoding(word, word_to_index):
  one_hot_vector = [0]*(len(word_to_index))
  index = word_to_index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

##### ② 원-핫 인코딩 : 형태소별 원-핫 벡터 출력

In [5]:
for word in tokens:
    print(word,"\t",one_hot_encoding(word, word_to_index))

나 	 [1, 0, 0, 0, 0, 0]
는 	 [0, 1, 0, 0, 0, 0]
자연어 	 [0, 0, 1, 0, 0, 0]
처리 	 [0, 0, 0, 1, 0, 0]
를 	 [0, 0, 0, 0, 1, 0]
배운다 	 [0, 0, 0, 0, 0, 1]


### 2. TF-IDF(빈도수 기반 텍스트/문서 벡터화)

#### 1) BoW(Bag of Words)

- BoW : 등장하는 단어들의 숫자를 세서 단어 주머니에 넣어 두는 것이다.

   <img src='images/bow.jpg' width='750px'>
      
   ###### * 출처 : https://web.stanford.edu/~jurafsky/slp3/4.pdf

- BoW를 만드는 과정
   - 각 단어에 고유한 인덱스를 부여한다.
   - 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.

##### ① 데이터 입력

In [6]:
corpus = ['If you do not walk today. you will have to run tomorrow']

##### ② 빈도 측정

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vector = CountVectorizer()
bow = vector.fit_transform(corpus)

print(bow.toarray())
print(vector.vocabulary_)

[[1 1 1 1 1 1 1 1 1 1 2]]
{'if': 2, 'you': 10, 'do': 0, 'not': 3, 'walk': 8, 'today': 6, 'will': 9, 'have': 1, 'to': 5, 'run': 4, 'tomorrow': 7}


##### ③ 불용어 제거

In [9]:
vector = CountVectorizer(stop_words='english')

##### ④ BoW

In [10]:
bow = vector.fit_transform(corpus)

print(bow.toarray())
print(vector.vocabulary_)

[[1 1 1 1]]
{'walk': 3, 'today': 1, 'run': 0, 'tomorrow': 2}


##### ⑤ 한국어 BoW

In [11]:
corpus = ['어리석은 자는 멀리서 행복을 찾고, 현명한 자는 자신의 발치에서 행복을 키워간다.']

In [12]:
vector = CountVectorizer()
bow = vector.fit_transform(corpus)

print(bow.toarray())
print(vector.vocabulary_)

[[1 1 1 2 1 1 1 2 1]]
{'어리석은': 2, '자는': 3, '멀리서': 0, '행복을': 7, '찾고': 5, '현명한': 8, '자신의': 4, '발치에서': 1, '키워간다': 6}


##### ⑥ 형태소 분석 후 BoW

In [13]:
import re
from konlpy.tag import Okt
okt = Okt()

tokens = okt.morphs(re.sub('[\,\.]','',corpus[0]))
print(tokens)

corpus1 = ' '.join(s for s in tokens)
corpus2 = []
corpus2.append(corpus1)
print(corpus2)

vector = CountVectorizer()
bow = vector.fit_transform(corpus2)

print(bow.toarray())
print(vector.vocabulary_)

['어리석은', '자는', '멀리', '서', '행복', '을', '찾고', '현명한', '자는', '자신', '의', '발치', '에서', '행복', '을', '키워', '간다']
['어리석은 자는 멀리 서 행복 을 찾고 현명한 자는 자신 의 발치 에서 행복 을 키워 간다']
[[1 1 1 1 1 2 1 1 1 2 1]]
{'어리석은': 3, '자는': 5, '멀리': 1, '행복': 9, '찾고': 7, '현명한': 10, '자신': 6, '발치': 2, '에서': 4, '키워': 8, '간다': 0}


#### 2) 문서 단어 행렬(Document-Term Matrix, DTM)

- 문서 단어 행렬(Document-Term Matrix)은 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것을 말한다.
- 각 문서에 대한 BoW를 하나의 행렬로 표현한 것이다.

   <img src='images/dtm.png' width='980px'>

##### ① 데이터 입력

In [14]:
docs = [
  '사고 싶은 스마트폰',
  '사고 싶은 스마트워치',
  '성능 좋은 스마트폰 스마트폰',
  '나는 스마트폰이 좋아요']

##### ② BoW

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
vector = CountVectorizer()
bow = vector.fit_transform(docs)

print(bow.toarray())
print(vector.vocabulary_)

[[0 1 0 0 1 0 1 0 0]
 [0 1 0 1 0 0 1 0 0]
 [0 0 1 0 2 0 0 0 1]
 [1 0 0 0 0 1 0 1 0]]
{'사고': 1, '싶은': 6, '스마트폰': 4, '스마트워치': 3, '성능': 2, '좋은': 8, '나는': 0, '스마트폰이': 5, '좋아요': 7}


##### ③ DTM

In [17]:
import pandas as pd

columns = []
for k, v in sorted(vector.vocabulary_.items(), key=lambda item:item[1]):
      columns.append(k)

df = pd.DataFrame(bow.toarray(), columns=columns)
df

,나는,사고,성능,스마트워치,스마트폰,스마트폰이,싶은,좋아요,좋은
0,0,1,0,0,1,0,1,0,0
1,0,1,0,1,0,0,1,0,0
2,0,0,1,0,2,0,0,0,1
3,1,0,0,0,0,1,0,1,0


#### 3) 어휘 빈도-문서 역빈도(Term Frequency-Inverse Docunment Frequency, TF-IDF) 분석

- TF-IDF : 단어의 빈도와 역 문서 빈도를 사용하여 DTM 내의 각 단어들 마다 중요한 정도를 가중치로 부여하는 것이다.
   - 단순히 빈도수가 높은 단어가 핵심어가 아닌, 특정 문서에서만 집중적으로 등장할 때 해당 단어가 문서의 주제를 잘 담고 있는 핵심어라고 가정한다.
   - 특정 문서에서 특정단어가 많이 등장하고 그 단어가 다른 문서에서 적게 등장할 때, 그 단어를 특정 문서의 핵심어로 간주한다.<br>
   → 특정 문서에서 특정 단어가 많이 등장하는 것을 의미한다.
   - 문서의 유사도 측정, 검색 시스템에서 검색 결과의 중요도 계산, 문서 내 특정 단어의 중요도 계산에 활용한다.
   - 어휘 빈도-문서 역빈도(tf-idf) : 어휘 빈도(tf)와 역문서 빈도(idf)를 곱해서 계산  $$ tf_{x,y} $$
   - 역문서 빈도(idf) : 다른 문서에서 등장하지 않는 단어 빈도를 의미  $$ log(N/df_x) $$      
   - 어휘 빈도-문서 역빈도(tf-idf) : 다음과 같이 표현  $$ W_{x,y} = tf_{x,y} * log(N/df_x) $$

- tf-idf 계산 방법

   <img src='images/tf-idf1.png' width='980px'>
   <img src='images/tf.png' width='980px'>
   <img src='images/idf.png' width='980px'>
   <img src='images/tf-idf2.png' width='980px'>

- tf-idf를 편리하게 계산하기 위해 `scikit-learn`의 `tfidfvectorizer`를 이용한다.
- 앞서 계산한 단어 빈도 수를 입력하여 tf-idf로 변환한다.

##### ① tfidf 객체 생성

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tfidf = TfidfVectorizer().fit(docs)

print(tfidf.transform(docs).toarray())
print(tfidf.vocabulary_)

[[0.         0.57735027 0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]
 [0.         0.52640543 0.         0.66767854 0.         0.
  0.52640543 0.         0.        ]
 [0.         0.         0.47212003 0.         0.7444497  0.
  0.         0.         0.47212003]
 [0.57735027 0.         0.         0.         0.         0.57735027
  0.         0.57735027 0.        ]]
{'사고': 1, '싶은': 6, '스마트폰': 4, '스마트워치': 3, '성능': 2, '좋은': 8, '나는': 0, '스마트폰이': 5, '좋아요': 7}


##### ② 시인성이 좋게 데이터프레임으로 변환

In [20]:
import pandas as pd

columns = []
for k, v in sorted(tfidf.vocabulary_.items(), key=lambda item:item[1]):
      columns.append(k)

df = pd.DataFrame(tfidf.transform(docs).toarray(), columns=columns)
df

,나는,사고,성능,스마트워치,스마트폰,스마트폰이,싶은,좋아요,좋은
0,0.00000,0.577350,0.00000,0.000000,0.57735,0.00000,0.577350,0.00000,0.00000
1,0.00000,0.526405,0.00000,0.667679,0.00000,0.00000,0.526405,0.00000,0.00000
2,0.00000,0.000000,0.47212,0.000000,0.74445,0.00000,0.000000,0.00000,0.47212
3,0.57735,0.000000,0.00000,0.000000,0.00000,0.57735,0.000000,0.57735,0.00000


### 3. 단어 임베딩(Word Embedding)

- 의미를 포함하는 단어를 벡터로 바꾸는 기법이며, 비슷한 분포를 가진 단어의 주변 단어들도 비슷한 의미를 가진다는 것을 가정한다.

- 예시)
   - 왼쪽 그림을 보면 왕과 여왕, 여왕과 여자가 같은 방향에 있다.
   - 의미가 비슷한 단어는 비슷한 방향에 위치하게 된다.
   - 단어 임베딩은 단어의 의미를 효과적으로 표현하기 때문에 one-hot encoding보다 학습 성능을 높일 수 있다.
   - 대량의 데이터로 단어 임베딩을 미리 학습시켜 두면, 문서 분류와 같은 과제에서 더 적은 데이터로도 학습된 임베딩을 사용하여 높은 성능을 낼 수 있다.

- Word Embedding
   
   <img src='images/w_embed1.png' width='980px'>

   ###### * 출처 : http://doc.mindscale.kr/km/unstructured/11.html

- 희소표현 : one-hot encoding은 단어의 의미를 전혀 고려하지 않으며 벡터의 길이가 총 단어 수가 되므로 매우 희박(sparse)한 형태가 된다.
- 밀집표현 : 이를 해결하기 위해 단어의 의미를 고려하여 좀 더 조밀한 차원에 단어를 벡터로 표현하는 것을 단어 임베딩(word embedding)이라 한다.
- 원핫 벡터로 표현된 단어를 밀집 벡터(dense vector)로 변환하는것을 워드 임베딩(word embedding)이라고 한다.
   - 이렇게 만들어진 단어 벡터는 단어의 의미 담고있으며, 단어 벡터 간의 연산도 가능하다.
   - 많은 양의 문서를 학습하여 얻어진 단어 벡터는 단어 간의 관계를 보다 정확하게 나타낸다.

- 10,000개의 단어로 이루어진 단어사전에서 희소표현과 밀집표현 비교

   <img src='images/w_embed2.png' width='980px'>

- Word2vec
   - 단어를 벡터로 임베딩하는 방식은 머신러닝을 통해 학습되는데, 신경망을 기반으로 한 단어 벡터화의 대표적 방법은 Word2Vec이다.
   - Word2vec은 단어 벡터 간 유의미한 유사도를 반영할 수 있도록 단어의 의미를 수치화 하는 방법이다.(문맥기반 학습)

- Word2vec 학습 방법 : CBOW(continuous bag of words)와 Skip-gram(SG)의 두 가지 알고리즘이 있고 일반적이다.
   
   <img src='images/word2vec1.png' width='980px'><br>

##### ① Word2Vec로 임베딩

In [21]:
from pptx import Presentation
prs = Presentation('data_set/리스크관리.pptx')

##### ② pptx 파일 읽어서 문자열로 추출

In [22]:
ppt_text = []

for slide in prs.slides:
    text_runs = []

    for shape in slide.shapes: 
        if not shape.has_text_frame:   
            continue   

        for paragraph in shape.text_frame.paragraphs: 

            for run in paragraph.runs:  
                text_runs.append(run.text)    
    
    ppt_text.append(" ".join(text_runs)) 

##### ③ 형태소 분석

In [23]:
from konlpy.tag import Okt
okt = Okt()

result = []

##### ④ 명사 추출

In [24]:
for txt in ppt_text:
    clean_ppt=[]	
    nouns = okt.nouns(txt)

    for noun in nouns:	
        if len(noun)!= 1:  
            clean_ppt.append(noun)
            
    result.append(clean_ppt)
print(result[0:5])

[['전략', '리스크', '관리', '방법론'], ['전략', '리스크', '관리', '이해', '개요', '리스크', '수익', '관리', '전략', '개요', '전략', '리스크', '관리', '방법론', '개요', '도입', '방안'], ['전략', '리스크', '관리', '이해', '비즈니스', '리스크', '소유', '범주', '모형', '비즈니스', '리스크', '소유', '이사회', '궁극', '리스크', '감독', '책임', '최상', '의사', '결정', '책임', '영진', '이사회', '감사', '위원회', '리스크', '관리', '내부감사', '리스크', '범주', '법무', '담당', '주주', '소비자', '공급', '파트너', '경쟁자', '시장', '신용', '재무', '구조', '보고', '프로세스', '인적', '자원', '설비', '지배', '구조', '경영', '계획', '대외', '협력', '재무', '재무', '보고', '대한', '내부통제', '노동', '외국', '노동자', '환경', '배기', '가스', '오염', '물질', '정책', '입법', '로비', '지적', '소유권', '의사결정', '지원', '정보', '기술'], ['전략', '리스크', '관리', '모델', '외부', '환경', '이해', '관계자', '프로세스', '리스크', '모니터', '리스크', '구체화', '리스크', '최적화', '전략', '리스크', '관리', '이해'], ['리스크', '관리', '전략', '리스크', '관리', '프로세스', '리스크', '최적화', '리스크', '구체화', '리스크', '판단', '리스크', '평가', '우선', '순위', '결정', '유형', '분류', '확률', '추정', '영향', '모형', '계량', '유형', '대응', '방안', '수립', '회피', '수용', '정비', '완화', '분석', '상관', '실행', '모니터링', '세련', '리드', '리스크', '평가', '리스크', '대응

##### ⑤ Word2Vec 훈련시키기 : Word2Vec 모델 활용

In [25]:
from gensim.models import Word2Vec

model = Word2Vec(result, size=50, window=2, min_count=1, workers=4, sg=0)

##### ⑥ 유사도가 높은 단어 추출

In [26]:
value = model.wv.most_similar('리스크', topn=5)
print(value)

[('위험', 0.9877815842628479), ('관리', 0.9862531423568726), ('수준', 0.9788869619369507), ('경영', 0.9787909388542175), ('기업', 0.9785916209220886)]


##### ⑦ 학습모델로 유사도 높은 단어 추출

In [27]:
model.wv.save_word2vec_format('word2')

In [28]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors 

##### ⑧ 유사도가 높은 단어 추출

In [29]:
load_model = KeyedVectors.load_word2vec_format('word2') 
load_model.most_similar('마케팅', topn=5)	

[('예방', 0.5825549960136414),
 ('영진', 0.556150496006012),
 ('달성', 0.5389859080314636),
 ('불확실', 0.531092643737793),
 ('내부', 0.5235405564308167)]